Let's begin with the code...

Downloading the Cats VS Dogs Dataset in a temperory zip folder.

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

Importing all the libraries required for the model to train.

In [0]:
import os
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop 

Unzipping the Zipped Dataset. The Dataset contains two sub-folders namely:

train
validation
Inside these both sub-folders, there are two another sub-folders named cats and dogs.

The train folder contains images of cats and dogs for training of the model.

The validation folder contains images of cats and dogs for validating the model.

In [0]:
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

Including 4 Convulotion layers with 32, 64, 128, 128 Convulotions respectively.

In [0]:
model = keras.models.Sequential([ keras.layers.Conv2D(32,(3,3), input_shape=(150,150,3), activation='relu'),
                                  keras.layers.MaxPool2D(2,2),
                                  keras.layers.Conv2D(64,(3,3), activation='relu'),
                                  keras.layers.MaxPool2D(2,2),
                                  keras.layers.Conv2D(128,(3,3),activation='relu'),
                                  keras.layers.MaxPool2D(2,2),
                                  keras.layers.Conv2D(128,(3,3),activation='relu'),
                                  keras.layers.MaxPool2D(2,2),
                                  keras.layers.Flatten(),
                                  keras.layers.Dense(128 , activation='relu'),
                                  keras.layers.Dense(1,activation='sigmoid')
])

In [0]:
model.compile(optimizer = RMSprop(lr=1e-4) , loss = 'binary_crossentropy' , metrics = ['accuracy'])

Keras's ImageDataGenerator helps in augmentation of the data.

In [0]:
train_datagen = ImageDataGenerator(rescale = 1/255 , rotation_range=0.2 , horizontal_flip=True, shear_range=0.2,
                                   width_shift_range= 0.2, height_shift_range= 0.2, zoom_range=0.2, fill_mode = 'nearest')
test_datagen = ImageDataGenerator(rescale=1/255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    batch_size = 20,
    class_mode='binary',
    target_size = (150,150)
)

test_gen = test_datagen.flow_from_directory(
    validation_dir,
    batch_size = 20,
    class_mode='binary',
    target_size = (150,150)
)

In [0]:
history = model.fit_generator(train_gen, steps_per_epoch=100, epochs=100, validation_data=test_gen, validation_steps=50 )

Plotting both Accuracy and Validation Accuracy and Loss and Validation Loss.

In [0]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)
  if classes>[0.5]:
    print(fn , "is a dog")
  else:
    print(fn , "is a cat")